# Print Metrics

## Creating Required Functions

### Import Library

In [43]:
from pathlib import Path
import os
import pandas as pd

### Function to Find the Best YOLO Model (best.pt)

In [44]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"[+] Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


### Functions to Load and Print Metrics from YOLO Training Results

In [45]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    print_final_metrics(df)


### Function to Compare Metrics Between Two YOLO Training Runs

In [46]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("Changes in Metrics Before and After Retraning:\n")
    print(f"{'Metric':<25} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 65)

    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        diff = val2 - val1
        if abs(diff) > 1e-6:
            # If increase, color green; if decrease, color red
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red

            # Printing with colors
            print(f"{label:<25} {val1:<10.5f} {val2:<10.5f} {diff:<10.5f} {trend}")


### Function to Compare mAP@0.5:0.95 Metrics Between Two Models (Before and After Retraining)

In [ ]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\nmAP@0.5:0.95 Differences Before and After Retraning:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 60)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        diff = map2 - map1
        if abs(diff) > 1e-6:
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green for increase
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red for decrease

            # Printing with colors
            print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {diff:<10.4f} {trend}")


### YOLO Model Evaluation and Metrics Extraction

In [ ]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model, yaml):
    return model.val(data=yaml)

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("[-] No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"[+] Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, yaml, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model, yaml)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)

### Per-Class mAP@0.5:0.95 Metrics Extraction and Display

In [49]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


## Calling Functions

### Before Retraning

In [50]:
yolov8 = './runs/train/yolov8'
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8/results.csv
Total number of epochs: 100

========== Final Training Metrics ==========
Training Loss: 1.890630
Precision: 0.567870
Recall: 0.301020
mAP@0.5: 0.316460
mAP@0.5:0.95: 0.202430

========== Final Validation Metrics ==========
Validation Loss: 5.499160


In [51]:
best_pt_path = find_best_model(yolov8)
evaluate_and_save_metrics(best_pt_path, yaml="yolov8.yaml",output_json_path="per_class_metrics.json")

[+] Found best.pt at: runs/train/yolov8/weights/best.pt
Ultralytics 8.3.109 🚀 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 OEM, 7957MiB)


Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/ssl49/Desktop/Automated-Labeling-for-Aerial-Images-main/Automated_SegmentAndYolo/datasets/new_dataset_yolo_split/val/labels.cache... 1181 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1181/1181 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:09<00:00,  7.84it/s]


                   all       1181      70393      0.516      0.298      0.327      0.214
                  pool          9          9      0.703      0.889      0.874      0.742
            vegetation         75       1780      0.533      0.266      0.271      0.144
                  roof         42         79     0.0482      0.722      0.528      0.429
                  wall         62        255      0.223      0.224      0.196      0.117
                window         38        139      0.612      0.324      0.361       0.22
                person         75        637      0.874      0.293      0.364       0.19
                   dog          6         12      0.286     0.0833      0.121     0.0591
                   car       1115      53669      0.822      0.283      0.486      0.243
               bicycle         32         53      0.431      0.302      0.335      0.141
                  tree         38        108      0.735      0.463      0.492      0.352
                 truc

In [52]:
print_per_class_metrics("per_class_metrics.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.2145         
pool            1          0.7424         
vegetation      2          0.1437         
roof            3          0.4295         
wall            4          0.1173         
window          5          0.2202         
person          6          0.1895         
dog             7          0.0591         
car             8          0.2427         
bicycle         9          0.1408         
tree            10         0.3524         
truck           11         0.036          
bus             12         0.0            
vehicle         13         0.1147         


### After Retraning

In [53]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8/results.csv
Total number of epochs: 10

========== Final Training Metrics ==========
Training Loss: 1.996340
Precision: 0.565270
Recall: 0.540430
mAP@0.5: 0.588980
mAP@0.5:0.95: 0.461290

========== Final Validation Metrics ==========
Validation Loss: 2.772980


In [54]:
best_pt_path = find_best_model(new_path)
evaluate_and_save_metrics(best_pt_path, yaml="yolo_retrain.yaml",output_json_path="per_class_metrics_retrain.json")

[+] Found best.pt at: runs/train/fine-tune-yolov8/weights/best.pt
Ultralytics 8.3.109 🚀 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 OEM, 7957MiB)
Model summary (fused): 72 layers, 3,008,378 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/ssl49/Desktop/Automated-Labeling-for-Aerial-Images-main/Automated_SegmentAndYolo/datasets/split_videos_dataset/val/labels.cache... 2101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2101/2101 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 132/132 [00:09<00:00, 13.97it/s]


                   all       2101       4219       0.46      0.616      0.589      0.467
                  pool          3          3      0.742          1      0.995      0.796
                  roof       1092       1541      0.754      0.913      0.931      0.786
                  wall        560        587      0.552      0.796      0.776      0.643
                window        108        117       0.25      0.592      0.381      0.271
                person        953       1451      0.589      0.642      0.635      0.496
               bicycle         83         88      0.187      0.217      0.169      0.112
                  tree        394        422      0.588      0.751      0.764      0.589
               vehicle         10         10     0.0196     0.0176     0.0572      0.046
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val15
[+] Saved per-class metrics to per_class_metrics_retrain.json


In [55]:
print_per_class_metrics("per_class_metrics_retrain.json")

Per-Class mAP@0.5:0.95 Metrics:

Class Name      Class ID   mAP@0.5:0.95   
----------------------------------------
unlabeled       0          0.4673         
pool            1          0.796          
vegetation      2          0.4673         
roof            3          0.7855         
wall            4          0.6431         
window          5          0.2713         
person          6          0.496          
dog             7          0.4673         
car             8          0.4673         
bicycle         9          0.1123         
tree            10         0.5887         
truck           11         0.4673         
bus             12         0.4673         
vehicle         13         0.046          


### Compare both metrics

In [56]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)

compare_final_metrics(results_csv_path_1, results_csv_path)

Changes in Metrics Before and After Retraning:

Metric                    Before     After      Diff       Trend
-----------------------------------------------------------------
Box Loss (Train)          0.65540    0.67524    0.01984     Increase
Cls Loss (Train)          0.39123    0.44638    0.05515     Increase
DFL Loss (Train)          0.84400    0.87472    0.03072     Increase
Precision                 0.56787    0.56527    -0.00260    Decrease
Recall                    0.30102    0.54043    0.23941     Increase
mAP@0.5                   0.31646    0.58898    0.27252     Increase
mAP@0.5:0.95              0.20243    0.46129    0.25886     Increase
Box Loss (Val)            2.04225    0.83381    -1.20844    Decrease
Cls Loss (Val)            2.38927    0.89746    -1.49181    Decrease
DFL Loss (Val)            1.06764    1.04171    -0.02593    Decrease


In [57]:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")


mAP@0.5:0.95 Differences Before and After Retraning:

Class           Before     After      Diff       Trend
------------------------------------------------------------
unlabeled       0.2145     0.4673     0.2528      Increase
pool            0.7424     0.7960     0.0536      Increase
vegetation      0.1437     0.4673     0.3236      Increase
roof            0.4295     0.7855     0.3560      Increase
wall            0.1173     0.6431     0.5258      Increase
window          0.2202     0.2713     0.0511      Increase
person          0.1895     0.4960     0.3065      Increase
dog             0.0591     0.4673     0.4082      Increase
car             0.2427     0.4673     0.2246      Increase
bicycle         0.1408     0.1123     -0.0285     Decrease
tree            0.3524     0.5887     0.2363      Increase
truck           0.0360     0.4673     0.4313      Increase
bus             0.0000     0.4673     0.4673      Increase
vehicle         0.1147     0.0460     -0.0687     Decrease
